In [49]:
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# configuarações de conexão com o banco de dados
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [34]:
# Carregando dados do Google Sheets
sheet_id = "1bKChKN304n76zBkZTfBxXrbSvSf2ZC4oXap940x8MNc"
sheet_name = "Relação Full x Tiny"
url = f"https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=xlsx"


df = pd.read_excel(url, sheet_name=sheet_name)

col = {
    "Código ML": "ml_inventory_id",
    "ID do anúncio": "ml_code",
    "ID Tiny": "tiny_id",
    "SKU": "ml_sku",
    "SKU Tiny": "tiny_sku",
    "Código Variação": "var_code",
    "Título do anúncio": "ad_title",
}

df.rename(columns=col, inplace=True)

df.shape

(877, 13)

In [35]:
cols_drop = [
    "Código universal",
    "Código Categoria",
    "Quantidade do item",
    "Unnamed: 10",
    "Unnamed: 11",
    "Unnamed: 12",
]
df = df.drop(cols_drop, axis=1)
df.sample()

,ml_sku,ml_inventory_id,ml_code,var_code,ad_title,tiny_sku,tiny_id
679,FULLTAGIMA27KCOMCAPA,DUCB06017,1557290709,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044


In [36]:
df.shape

(877, 7)

In [37]:
duplicatas = df.duplicated()
numero_duplicatas = duplicatas.sum()
print(f"Número de duplicatas: {numero_duplicatas}")

df_duplicatas = df[df.duplicated()]
print("Linhas duplicadas:")
df_duplicatas

Número de duplicatas: 5
Linhas duplicadas:


,ml_sku,ml_inventory_id,ml_code,var_code,ad_title,tiny_sku,tiny_id
680,FULLTAGIMA27KCOMCAPA,DUCB06017,1557290709,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044
682,FULLTAGIMA27KCOMCAPAC,SSBF06040,1557306058,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044
718,FULLC4RH16,IYUQ71257,2192258271,Preto,Suporte Violão Guitarra Baixo Parede Resistent...,7891088100057,737198870
795,CTX3000SIMPLES,KVWJ40778,1233445054,Preto,Teclado Musical Profissional Arranjador Digit ...,4971850349099,506979980
798,FULLORIONSPX15HH,WXUT79339,1512798100,NaN,Prato Bateria Orion Chimbal Hihat 15 Spx15hh X...,7898608740991,681951269


In [38]:
x = 1557290709
lines = df[df["ml_code"] == x]
line = df_duplicatas[df_duplicatas["ml_code"] == x]
lines

,ml_sku,ml_inventory_id,ml_code,var_code,ad_title,tiny_sku,tiny_id
679,FULLTAGIMA27KCOMCAPA,DUCB06017,1557290709,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044
680,FULLTAGIMA27KCOMCAPA,DUCB06017,1557290709,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044


In [39]:
line

,ml_sku,ml_inventory_id,ml_code,var_code,ad_title,tiny_sku,tiny_id
680,FULLTAGIMA27KCOMCAPA,DUCB06017,1557290709,Marrom,Ukulelê Tenor Profissional Tagima 27k Mahogany...,7898563978156,512617044


In [40]:
df = df.drop_duplicates()
df.shape

(872, 7)

In [52]:
# Ela contém a relação entre tiny_id e inventory_id do ML
# e não permite pares de valores duplicados entre (ml_inventory_id, tiny_id)

conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df.iterrows():
    insert_query = sql.SQL(
        "INSERT INTO tiny_fulfillment (ml_sku, ml_inventory_id, ml_code, var_code, ad_title, tiny_sku, tiny_id) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(
        insert_query,
        (
            row["ml_sku"],
            row["ml_inventory_id"],
            row["ml_code"],
            row["var_code"],
            row["ad_title"],
            row["tiny_sku"],
            row["tiny_id"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


In [44]:
df

,ml_sku,ml_inventory_id,ml_code,var_code,ad_title,tiny_sku,tiny_id
0,FULLNI5BM,DSGP06967,1992567302,1 Par 5B,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,7897937421021,735947207
1,FULLNI7AM,DSGP06979,1992567302,1 Par 7A,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,7897937421007,747848159
2,FULLNI5AM,JJSZ06277,1992567302,1 Par 5A,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,7897937421014,698412654
3,FULLVANDOREN2CR1015,DGCL82617,2606294854,"1,5","2 Palheta Clarinete Vandoren Tradicional 1 1,5...",7897626822009,749528560
4,FULLVANDOREN2CR102,GCYF86172,2606294854,2,"2 Palheta Clarinete Vandoren Tradicional 1 1,5...",7897626822016,749530945
...,...,...,...,...,...,...,...
872,MD18SBKIT1PL1,NaN,980377955,NaN,Violão Elétrico Aço Tagima Memphis Md-18 Sunbu...,Palheta,565665071
873,7643D,OYUK51882,1363602423,NaN,Bocal Para Trompete Dolphin 7c Prateado Original,7897626876439,509636083
874,FULLBASSOVTSL52,BLHH53768,2004836143,NaN,Correia Guitarra Violão Baixo Basso Alça Couro...,7898529535775,729127557
875,FULLBASSOECOBASS02,NaN,2006362042,NaN,Correia Para Contra Baixo Basso 9 Cm Eco Couro...,7898529535720,729130774
